In [ ]:
elastic_rods_dir = '../../../../elastic_rods/python/'
weaving_dir = '../../../'
import os
import os.path as osp
import sys; sys.path.append(elastic_rods_dir); sys.path.append(weaving_dir)
import numpy as np, elastic_rods, linkage_vis
import numpy.linalg as la
from bending_validation import suppress_stdout as so
import matplotlib.pyplot as plt
from elastic_rods import EnergyType, InterleavingType

# weaving
import analysis_helper, ribbon_linkage_helper, mesh_vis, linkage_utils, compute_curve_from_curvature, pipeline_helper, importlib
importlib.reload(analysis_helper)
importlib.reload(ribbon_linkage_helper)
importlib.reload(mesh_vis)
importlib.reload(linkage_utils)
importlib.reload(compute_curve_from_curvature)
importlib.reload(pipeline_helper)
from analysis_helper import (compare_turning_angle,
                            is_on_sphere, 
                            get_distance_to_center_scalar_field, 
                            plot_curvatures, 
                            get_curvature_scalar_field,
                            construct_elastic_rod_loop_from_rod_segments, 
                            concatenate_rod_properties_from_rod_segments, 
                            compute_min_distance_rigid_transformation)
from ribbon_linkage_helper import (update_rest_curvature, 
                                   set_ribbon_linkage,
                                   export_linkage_geometry_to_obj,
                                   write_linkage_ribbon_output_florin,
                                   write_centerline_normal_deviation_to_linkage_mesh)

from compute_curve_from_curvature import (match_geo_curvature_and_edge_len, get_all_curve_pattern)
from linkage_utils import order_segments_by_ribbons, get_turning_angle_and_length_from_ordered_rods

from pipeline_helper import (initialize_linkage, get_normal_deviation, set_joint_vector_field, stage_1_optimization, initialize_stage_2_optimizer, stage_2_optimization, InputOrganizer, write_all_output, set_surface_view_options, get_structure_analysis_view, get_max_distance_to_target_surface, contact_optimization, show_selected_joints, highlight_rod_and_joint)

import vis.fields
import matplotlib.cm as cm
import time

In [ ]:
import parallelism
parallelism.set_max_num_tbb_threads(12)
parallelism.set_hessian_assembly_num_threads(4)
parallelism.set_gradient_assembly_num_threads(4)

In [ ]:
scale = 1
rod_length = 0.07244622391567243 * scale
width = rod_length / 20 * 10 
thickness = width / 10 * 0.35 
print(width, thickness)
io = InputOrganizer('kleinbottle_projected_{}'.format(scale), thickness, width, weaving_dir)

In [ ]:
import py_newton_optimizer
OPTS = py_newton_optimizer.NewtonOptimizerOptions()
OPTS.gradTol = 1e-6
OPTS.verbose = 1;
OPTS.beta = 1e-8
OPTS.niter = 200
OPTS.verboseNonPosDef = False
rw = 1
sw = 10
drw = 0.1
dsw = 0.1

In [ ]:
# vf = show_selected_joints(curved_linkage, [30, 205, 91, 303, 525, 462, 496, 152, 425, 254, 391, 348, 151, 255, 255, 349, 349, 31, 92, 461, 92, 461, 151])
# curved_linkage_view.update(vectorField = vf)

In [ ]:
with so(): curved_linkage = initialize_linkage(surface_path = io.SURFACE_PATH, useCenterline = True, model_path = io.MODEL_PATH, cross_section = io.RIBBON_CS, subdivision_res = io.SUBDIVISION_RESOLUTION, interleaving_type=InterleavingType.weaving, use_constant_width = False, width_scale = [0.5, 1.5])
curved_linkage.set_design_parameter_config(use_restLen = True, use_restKappa = False)
curved_save_tgt_joint_pos = curved_linkage.jointPositions();
curved_linkage_view = linkage_vis.LinkageViewerWithSurface(curved_linkage, io.SURFACE_PATH)
set_surface_view_options(curved_linkage_view)
curved_linkage_view.show()

In [ ]:
curved_linkage.jointPositions()

In [ ]:
dof = np.load('klein_bottle_straight_ribbon_dof_020720.npy')

In [ ]:
dof.shape

In [ ]:
curved_linkage.numExtendedDoFPSRL()

In [ ]:
curved_linkage.setExtendedDoFsPSRL(np.load('klein_bottle_straight_ribbon_dof_020720.npy'))

In [ ]:
curved_linkage.attraction_weight = 1e-5

In [ ]:
# iterateData = stage_1_optimization(curved_linkage, rw, sw, curved_linkage_view)

In [ ]:
def eqm_callback(prob, i):
    curved_linkage_view.update()

In [ ]:
OPTS.niter = 10000

In [ ]:
elastic_rods.compute_equilibrium(curved_linkage, callback = eqm_callback, options = OPTS)

In [ ]:
structure_view = get_structure_analysis_view(curved_linkage)
structure_view.show()

In [ ]:
curved_linkage.energy()

In [ ]:
# contact_optimization(optimizer, curved_linkage_view, -1, 1e4)

In [ ]:
deviation, deviation_vector, joint_normals, joint_projection_normals = get_normal_deviation(curved_linkage)

In [ ]:
set_joint_vector_field(curved_linkage, curved_linkage_view, deviation_vector)

In [ ]:
deviation_angle = write_centerline_normal_deviation_to_linkage_mesh(curved_linkage, 'kleinbottle_straight_ribbon')

In [ ]:
linkage = curved_linkage
threshold = np.mean(io.RIBBON_CS)
# def write_centerline_normal_deviation_to_linkage_mesh(linkage, width, model_name):
def get_color_scheme(colors):
    cmap = plt.cm.plasma
#     cmap = cmap2
    return cmap(colors)
[points, faces, normals] = linkage.visualizationGeometry()
new_normals = np.zeros_like(points)
compare_points = []
point_to_compare_point_map = -1 * np.ones(len(points))
compare_pt_index = 0
for face in faces:
    pt1, pt2, pt3 = points[face[0]], points[face[1]], points[face[2]]
    e12 = la.norm(pt1 - pt2)
    e13 = la.norm(pt1 - pt3)
    e23 = la.norm(pt2 - pt3)
    if np.max([e12, e13, e23]) > threshold:
        compare_pt = (pt1 + pt2 + pt3)/3
        compare_points.append(compare_pt)
        face_normal = np.cross(pt1-pt2, pt1-pt3)
        face_normal /= la.norm(face_normal)
        new_normals[face[0]] = face_normal
        new_normals[face[1]] = face_normal
        new_normals[face[2]] = face_normal
        point_to_compare_point_map[face[0]] = compare_pt_index
        point_to_compare_point_map[face[1]] = compare_pt_index
        point_to_compare_point_map[face[2]] = compare_pt_index
        compare_pt_index += 1

# linkage_centerline_normals = np.array(linkage.visualizationGeometry()[2])
# faces = np.array(linkage.visualizationGeometry()[1])
# for faces in 
# linkage_centerline_projection_normals = np.array(linkage.get_closest_point_normal((linkage.visualizationGeometry()[0]).flatten()))
# linkage_centerline_projection_normals = linkage_centerline_projection_normals.reshape(linkage_centerline_normals.shape)
# deviation_angle = []
# for i in range(len(linkage_centerline_normals)):
#     closeness = np.dot(linkage_centerline_projection_normals[i], linkage_centerline_normals[i])
#     # The value is between 0 and pi/2; scale to between 0 and 1
#     angle = min(np.arccos(abs(closeness))/ (np.pi/2), 1)
#     if (1-angle) < 0.4:
#         angle = 0
#     if (angle > 1 or angle < 0):
#         print("Wrong normal deviation angle!")
#     deviation_angle.append(angle)
# print("max deviation angle: ", max(deviation_angle))
# deviation_angle = np.array(deviation_angle) / max(deviation_angle)
# export_linkage_geometry_to_obj(linkage, 'normal_deviation_mesh_{}.obj'.format(model_name), vd = deviation_angle, use_color=True, colors = get_color_scheme(deviation_angle))
# return deviation_angle

In [ ]:
compare_points = np.array(compare_points)

In [ ]:
new_normals

In [ ]:
point_to_compare_point_map[3]

In [ ]:
linkage_centerline_projection_normals = np.array(linkage.get_closest_point_normal(compare_points.flatten()))
linkage_centerline_projection_normals = linkage_centerline_projection_normals.reshape(compare_points.shape)
deviation_angle = []
for i in range(len(new_normals)):
    if la.norm(new_normals[i]) == 0:
        deviation_angle.append(0)
    else:
        closeness = np.dot(linkage_centerline_projection_normals[int(point_to_compare_point_map[i])], new_normals[i])
        # The value is between 0 and pi/2; scale to between 0 and 1
        angle = min(np.arccos(abs(closeness))/ (np.pi/2), 0.6)
        if (angle > 1 or angle < 0):
            print("Wrong normal deviation angle!")
        deviation_angle.append(angle)
print("max deviation angle: ", max(deviation_angle))
deviation_angle = np.array(deviation_angle) / max(deviation_angle)
export_linkage_geometry_to_obj(linkage, 'normal_deviation_mesh_{}.obj'.format(io.MODEL_NAME), vd = deviation_angle, use_color=True, colors = get_color_scheme(deviation_angle))
# return deviation_angle

In [ ]:
plt.plot(np.sort(deviation_angle))

In [ ]:
os.chdir(save_path)

In [ ]:
save_path = os.getcwd()
write_all_output(curved_linkage, io.SUBDIVISION_RESOLUTION, 'kleinbottle_straight_ribbon', io.RIBBON_NAME, False, 100, min(io.RIBBON_CS), target_width = 5, flip_angles = False, width = max(io.RIBBON_CS))
os.chdir(save_path)